### All we need is to ensure that our model evaluation output of all loaders are the same so that we could easily compare the models. Currenlty we do not have any proper models, this is quite the problem you see. 

### Should we iterate over the indexes - yes. We can compare the RNN and FNN - with the FNN. So many different things we wanted to plug.

In [1]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
import logging as log
from utils.preprocessing import Shaper, MinMaxScaler, minmax_scale
from utils.configs import BaseConf

In [2]:
import unittest
# from datasets.flat_dataset import FlatDataset
from utils.preprocessing import Shaper
from utils.mock_data import mock_data
import numpy as np
import os
import matplotlib.pyplot as plt
from pprint import pprint

In [3]:
pprint(os.listdir('./data/raw'))

['tract2points.pkl',
 'crimes.pkl',
 'point2feats_res18.pkl',
 'openweatherdata.json',
 'weather_minmax_normed.pkl',
 'tract2km2.pkl',
 'point2tract.pkl',
 'point2spatial_info.pkl',
 'census.pkl',
 'resnet18_state_dict.pkl',
 'valid_tracts.npy',
 'crimes_2012_to_2018.pkl',
 'census_minmax_normed.pkl',
 'valid_points.npy',
 'street_view_coords.npy',
 'tract_boundaries.pkl',
 'invalid_tracts.npy']


In [4]:
crimes = pd.read_pickle('./data/raw/crimes_2012_to_2018.pkl')

In [6]:
crimes.tail()

,ID,Date,Primary Type,Arrest,Latitude,Longitude,tract
145308,11552731,2018-12-31 23:55:00,BATTERY,False,41.751914,-87.647717,7108.0
145307,11552724,2018-12-31 23:56:00,BATTERY,True,41.857068,-87.657625,8429.0
145306,11552699,2018-12-31 23:57:00,CRIMINAL DAMAGE,False,41.740521,-87.647391,7110.0
145305,11561837,2018-12-31 23:59:00,DECEPTIVE PRACTICE,False,41.763181,-87.657709,6719.0
145304,11556487,2018-12-31 23:59:00,CRIMINAL DAMAGE,False,41.689079,-87.696064,7401.0


In [42]:
from sys import getsizeof
def size(a):
    print(f"{getsizeof(a)/1000} kB")
import torch

In [108]:
npdata = np.random.binomial(1,0.1,(300,30,30,30))
# npdata = np.random.randn(300,30,30,30)
dense_data = torch.Tensor(npdata)
spare_data = dense_data.to_sparse()
size(npdata)
size(dense_data)
size(spare_data)


np.savez_compressed("000_numpy_tensor.npz", npdata)
# torch.save(spare_data, "000_spare_data.pth")
torch.save(dense_data, "000_dense_data.pth")

64800.144 kB
0.072 kB
0.072 kB


# Todo `AUC` over time and if re-training is needed
- add noise to tragets
- get auc for

In [118]:
data_dim_str = "T24H-X850M-Y880M"  #"T1H-X1700M-Y1760M"  # needs to exist
data_dim_str = data_dim_str + "_2013-01-01_2015-01-01"
model_name = "RNN-CRIME-MODEL"  # needs to be created

data_path = f"./data/processed/{data_dim_str}/"
model_path = f"{data_path}models/{model_name}/"
os.makedirs(data_path, exist_ok=True)
os.makedirs(model_path, exist_ok=True)

'24H'

In [182]:
import pickle
from pprint import pformat
with np.load(f"{data_path}generated_data.npz") as zip_file:
    log.info("Data shapes of files in generated_data.npz")
    for k, v in zip_file.items():
        try:
            log.info(f"\t{k} shape {np.shape(v)}")
        except Exception as e:
            log.info(f"\t{k} ")
            
      
        
t_range = pd.read_pickle(data_path + "t_range.pkl")
log.info(f"\tt_range shape {np.shape(t_range)}")

from utils.plots import im

In [162]:
data = np.random.binomial(1,0.1,(5,10,10))
data = np.expand_dims(data,axis=1)


shaper = Shaper(data)
s = shaper.squeeze(data)

with open("shaper.pkl", "wb") as shaper_file:
    pickle.dump(shaper, shaper_file)   


with open("shaper.pkl", "rb") as shaper_file:
    new_shaper = pickle.load(shaper_file)  
    
print(s.shape)    
unsqueezed_data = new_shaper.unsqueeze(s)

print(np.equal(unsqueezed_data, data).all())
print(id(unsqueezed_data) == id(data))

(5, 1, 45)
True
False


# Sequenced RNN

In [ ]:

# data_dim_strs = [
#     "T12H-X850M-Y880M/",
#     "T1H-X1700M-Y1760M/",
#     "T24H-X425M-Y440M/",
#     "T24H-X850M-Y880M/",
#     "T24H-X85M-Y110M/",
#     "T3H-X850M-Y880M/",
#     "T4H-X850M-Y880M/",
#     "T6H-X850M-Y880M/"
# ]

data_dim_strs = os.listdir("./data/processed")[1:]
pprint(data_dim_strs)

In [ ]:
for data_dim_str in data_dim_strs:
    print(data_dim_str)
    data_path = f"./data/processed/{data_dim_str}/"
    zip_file = np.load(data_path + "generated_data.npz")

    # print info on the read data
    log.info("Data shapes of files in generated_data.npz")
    for k in zip_file.keys():
        try:
            v = zip_file[k]
            print(f"\t{k} shape {np.shape(v)}")
        except ValueError as e:
            print(f"Issue loading {k} ->{e}")
            
        
    t_range = pd.read_pickle(data_path + "t_range.pkl")
    print(f"\tt_range shape {np.shape(t_range)}")

In [ ]:
for k in zip_file.keys():
    print(k)

In [ ]:
data_dim_str = "T24H-X850M-Y880M"
data_path = f"./data/processed/{data_dim_str}/"


conf_dict = {
    "seed": 3,
    "resume": False,
    "early_stopping": False,
    "use_cuda": False,
    "val_ratio": 0.1,
    "tst_ratio": 0.2,
    "sub_sample_train_set": True,
    "sub_sample_validation_set": True,
    "sub_sample_test_set": False,
    "flatten_grid": True,
    "lr": 1e-3,
    "weight_decay": 1e-8,
    "max_epochs": 10,
    "batch_size": 64,
    "dropout": 0,
    "shuffle": False,
    "num_workers": 6,
    "seq_len": 0,
    "top_k_cells": 20,
    "use_top_k_cells": False,
}


conf = BaseConf(conf_dict=conf_dict)

data_group = FlatDataGroup(data_path=data_path, conf=conf)

In [ ]:
test_set = data_group.testing_set
# get evaluation of each time step - like evaluate sub-test sets per time and see if the actual accuracy dies off.



In [ ]:
test_set.targets.shape,test_set.crimes.shape

In [ ]:
test_set.targets.shape

# Try simple crime info only training loop
### Check other training loops - already have good examples with checkpointing set up

## Useful plotting - when it comes to the per cell level predictions
## TODO: add cell index sorting to the dataset/datagroup

In [ ]:
og_crimes = data_group.crime_scaler.inverse_transform(data_group.crimes)
og_crimes = np.exp2(og_crimes) - 1

In [ ]:
top_k = 10
last_n_steps = 150
crimes = og_crimes[:,0]
s = data_group.shaper.squeeze(crimes)
indices = np.argsort(s.mean(0))
sorted_s = s[:last_n_steps,indices[-top_k:]]
plt.figure(figsize=(20,20))
plt.imshow(sorted_s,cmap='viridis')
plt.colorbar()
plt.show()

im(crimes.mean(0))

In [ ]:
for i in range(2):
    data = sorted_s[:,-(i+1)]
    ones = np.copy(data)
    ones[ones == -1]
    mean = np.ones_like(data)*np.mean(data)
    cell_plotter = CellPlotter("test")
    cell_plotter.plot_predictions(y_true=[1,0,0,1,0,0,1,0,0,1],
                                  y_pred=None,[0,1,0,0,1,0,1,0,0,1],
                                  probas_pred=[.1,.01,.06,.6,0,.3,.8,0,0,.1])
    cell_plotter.show()